In [5]:
import torch
import sys
import matplotlib.pyplot as plt
sys.path.append("C:/Users/user/NeuroSegNet/")
from monai.transforms import LoadImage, Resize
from src.xai.gradcam import apply_gradcam
from src.models.neurosegnet import NeuroSegNet
import yaml
from pathlib import Path
import numpy as np

# Load config and model
with open("C:/Users/user/NeuroSegNet/configs/model/neurosegnet.yml") as f:
    config = yaml.safe_load(f)

model = NeuroSegNet(config)
checkpoint = torch.load("experiments/neurosegnet_v1/checkpoints/best_model.pth", map_location="cpu")
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


KeyError: 'model_state_dict'

In [ ]:
# Load a validation case
image_path = sorted(Path("data/val_preprocessed/").glob("*.nii.gz"))[0]
loader = LoadImage(image_only=True)
img_np = loader(str(image_path))  # [H,W,D,C]
img_np = np.moveaxis(img_np, -1, 0)  # [C,H,W,D]
input_tensor = torch.tensor(img_np, dtype=torch.float32).unsqueeze(0).to(device)

# Apply GradCAM
target_layer = model.encoder.layers[-1]
cam = apply_gradcam(model, input_tensor, target_layer, resize_to=(128,128,128))

# Visualize mid-slice
plt.figure(figsize=(8,4))
mid_slice = cam.shape[2] // 2
plt.imshow(cam[:, :, mid_slice], cmap='hot')
plt.title("Grad-CAM (Mid Slice)")
plt.axis("off")
plt.show()